In [1]:
import os

from main import FeverLoader, PubhealthLoader, ClimateFeverLoader

## Load Data

In [2]:
root = '../data_2023_06_02'

climate_in = os.path.join(root, 'preprocessed/CLIMATE-FEVER')
pubhealth_in = os.path.join(root, 'preprocessed/PUBHEALTH')
fever_in = os.path.join(root, 'preprocessed/FEVER')

In [3]:
fever_train_ds, fever_dev_ds, fever_test_ds = FeverLoader.load(fever_in)
print(len(fever_test_ds))
print(len(fever_dev_ds))
print(len(fever_train_ds))

9999
9999
145449


In [4]:
pubhealth_train_ds, pubhealth_dev_ds, pubhealth_test_ds = PubhealthLoader.load(pubhealth_in)
print(len(pubhealth_train_ds))
print(len(pubhealth_dev_ds))
print(len(pubhealth_test_ds))

8370
1050
1032


In [5]:
climate_ds = ClimateFeverLoader.load(climate_in)
print(len(climate_ds))

1381


## Training

In [11]:
model_name = "bert-base-uncased" #https://huggingface.co/bert-base-uncased
# model = "bert-large-uncased" #https://huggingface.co/bert-large-uncased

#### Set up training data
- combine fever and climate data. Infer on pubhealth data

In [12]:
from datasets import Dataset, DatasetDict, ClassLabel, Value, Features

In [13]:
ds = DatasetDict()
features = Features({
    "claim": Value("string"), 
    "evidence": Value("string"),
    "label": ClassLabel(num_classes=3, names=["SUPPORTS", "REFUTES", "NOT ENOUGH INFO"])
})

In [18]:
#split climate data
from sklearn.model_selection import train_test_split

climate_train_ds, climate_dev_ds = train_test_split(climate_ds, random_state=392)
print(len(climate_train_ds))
print(len(climate_dev_ds))

1035
346


In [19]:
ds['train'] = Dataset.from_list(fever_train_ds + climate_train_ds, features=features)
ds['validation'] = Dataset.from_list(fever_dev_ds + climate_dev_ds, features=features)
ds['fever_test'] = Dataset.from_list(fever_test_ds, features=features)
ds['pubhealth_test'] = Dataset.from_list(pubhealth_test_ds, features=features)
# ds['climate']  = Dataset.from_list(climate_ds, features=features)

#### Tokenize data

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess_function(samples):
    return tokenizer(samples['claim'], samples['evidence'], 
                     padding=True,
                     truncation='only_second')

In [21]:
encoded_ds = ds.map(preprocess_function, batched=True)

#### Setup model

In [22]:
from transformers import AutoModelForSequenceClassification

In [23]:
num_labels = 3 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
# print(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#### Setup trainer

In [24]:
import numpy as np
import evaluate 
from transformers import TrainingArguments, Trainer

In [25]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references = labels)

In [26]:
batch_size = 16 #defaults to 8
num_epochs = 5

args = TrainingArguments(
    "../models/FEVER_BERT_FEVER_CLIMATE",
    evaluation_strategy = "epoch",
    # evaluation_strategy = "steps",
    save_strategy = "epoch",
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    load_best_model_at_end = True,
    # metric_for_best_model = "accuracy"
)   

In [27]:
trainer = Trainer(
    model, 
    args,
    train_dataset = encoded_ds["train"],
    eval_dataset = encoded_ds["validation"],
    tokenizer = tokenizer, 
    compute_metrics = compute_metrics
)

#### Train model

In [28]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: evidence, claim. If evidence, claim are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 146484
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 11445
  Number of trainable parameters = 109484547
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using

Epoch,Training Loss,Validation Loss,Accuracy
1,0.146800,0.207794,0.930981
2,0.098900,0.230082,0.929821
3,0.069600,0.228196,0.942484
4,0.043300,0.247676,0.944611
5,0.028200,0.287810,0.944514


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: evidence, claim. If evidence, claim are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10345
  Batch size = 64
Saving model checkpoint to ../models/FEVER_BERT_FEVER_CLIMATE/checkpoint-2289
Configuration saved in ../models/FEVER_BERT_FEVER_CLIMATE/checkpoint-2289/config.json
Model weights saved in ../models/FEVER_BERT_FEVER_CLIMATE/checkpoint-2289/pytorch_model.bin
tokenizer config file saved in ../models/FEVER_BERT_FEVER_CLIMATE/checkpoint-2289/tokenizer_config.json
Special tokens file saved in ../models/FEVER_BERT_FEVER_CLIMATE/checkpoint-2289/special_tokens_map.json
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scal

TrainOutput(global_step=11445, training_loss=0.08425184727340038, metrics={'train_runtime': 5080.6592, 'train_samples_per_second': 144.158, 'train_steps_per_second': 2.253, 'total_flos': 1.9270952941234176e+17, 'train_loss': 0.08425184727340038, 'epoch': 5.0})

In [29]:
trainer.evaluate(encoded_ds['fever_test'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: evidence, claim. If evidence, claim are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9999
  Batch size = 64
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.19679760932922363,
 'eval_accuracy': 0.9362936293629363,
 'eval_runtime': 27.1176,
 'eval_samples_per_second': 368.727,
 'eval_steps_per_second': 5.79,
 'epoch': 5.0}

In [30]:
trainer.evaluate(encoded_ds['pubhealth_test'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: evidence, claim. If evidence, claim are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1032
  Batch size = 64
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.1466269493103027,
 'eval_accuracy': 0.5513565891472868,
 'eval_runtime': 2.9098,
 'eval_samples_per_second': 354.669,
 'eval_steps_per_second': 5.842,
 'epoch': 5.0}

## Inference

In [ ]:
# load best model
best_model_checkpoint = "../models/FEVER_BERT_FEVER_CLIMATE/checkpoint-2289"